In [1]:
import os
from collections import Counter

# Create n-grams from a text
# n=2 (with padding " hello world "):
# [' ', ' h', 'he', 'el', 'll', 'lo', 'o ', ' w', 'wo', 'or', 'rl', 'ld', 'd ']
def create_ngrams(text, size):
    # TEXT CLEANING
    # 1. Remove non-alphabetic characters
    # 2. Convert to lowercase
    # 3. Remove extra spaces
    # Example: "Hello, World!" becomes "hello world".
    text = ''.join([char.lower() for char in text if char.isalpha() or char.isspace()])
    text = ' '.join(text.split())
    all_ngrams = []
    for n in size:
        # Add padding to the text to capture beginning and ending n-grams
        padded_text = (' ' * (n-1)) + text + (' ' * (n-1))
        all_ngrams.extend([padded_text[i:i+n] for i in range(len(padded_text)-n+1)])
    return all_ngrams

def count_ngram_frequencies(ngrams):
    return Counter(ngrams)

# Reads a text file, generate n-grams of specified sizes, count their frequencies, and return the frequency count.
def analyze_text_file(filepath, size):
    try:
        with open(filepath, 'r', encoding='utf-8') as file:
            text = file.read()
        ngrams = create_ngrams(text, size)
        ngram_frequencies = count_ngram_frequencies(ngrams)
        return ngram_frequencies
    except Exception as e:
        print(f"Error reading file {filepath}: {e}")
        return None

# Save language profiles to individual text files in the specified output directory.
def store_language_profiles(profiles, output_folder):
    os.makedirs(output_folder, exist_ok=True)

    # Iterate over each language and its corresponding profile
    for lang, profile in profiles.items():
        filename = os.path.join(output_folder, f"{lang}_profile.txt")
        with open(filename, 'w', encoding='utf-8') as file:
            
            # Write each n-gram and its frequency to the file
            for ngram, frequency in profile.items():
                file.write(f"{ngram} -- {frequency}\n")
        
        print(f"Saved {lang} profile")

def load_language_profile(filepath):
    """Load a language profile from a file."""
    profile = Counter()
    with open(filepath, 'r', encoding='utf-8') as file:
        for line in file:

            parts = line.split(' -- ')
            if len(parts) == 2:
                ngram, count = parts
                if ngram == '':
                    ngram = ' '  # Handle the special case where ngram is a space
                try:
                    profile[ngram] = int(count)
                except ValueError:
                    print(f"Skipping line with non-integer count: {line.strip()}")
            else:
                print(f"Skipping improperly formatted line: {line.strip()}")
    return profile

def load_all_profiles(profiles_directory, languages):
    """Load all language profiles from the specified directory."""
    profiles = {}
    for language in languages:
        filepath = os.path.join(profiles_directory, f"{language}_profile.txt")
        profiles[language] = load_language_profile(filepath)
    return profiles

# Define n-gram values
n = [1, 2, 3, 4, 5]

# Define language files
language_files = {
    'English': 'english.txt',
    'Slovenian': 'slovenian.txt',
    'German': 'german.txt',
    'Spanish': 'spanish.txt',
    'Croatian': 'croatian.txt',
    'Turkey': 'turkish.txt',

}

# Load all language profiles
language_profiles = load_all_profiles("C:/Program Files/Visual Studio Code Projects/Language Technology Lab Works/Lab 3/profiles", language_files.keys())

# Print out the loaded profiles for verification
for language, profile in language_profiles.items():
    print(f"{language} profile loaded with {len(profile)} n-grams.")


English profile loaded with 300 n-grams.
Slovenian profile loaded with 300 n-grams.
German profile loaded with 300 n-grams.
Spanish profile loaded with 300 n-grams.
Croatian profile loaded with 300 n-grams.
Turkey profile loaded with 300 n-grams.


In [2]:
def compute_text_profile(text, n):
    """Compute n-gram profile for given text."""
    ngrams = create_ngrams(text, n)
    ngram_counts = count_ngram_frequencies(ngrams)
    return ngram_counts

# Example text input
example_text = "This is a sample text to test the n-gram profile generation."
example_ngram_profile = compute_text_profile(example_text, n)
print("N-gram profile for example text:", example_ngram_profile.most_common(30))


N-gram profile for example text: [(' ', 10), ('t', 8), ('e', 7), (' t', 5), ('i', 4), ('s', 4), ('a', 4), ('o', 3), ('n', 3), ('r', 3), ('e ', 3), ('h', 2), ('m', 2), ('p', 2), ('l', 2), ('g', 2), ('th', 2), ('is', 2), ('s ', 2), ('am', 2), ('le', 2), ('te', 2), ('t ', 2), ('ra', 2), (' th', 2), ('is ', 2), ('le ', 2), (' te', 2), ('t t', 2), ('x', 1)]


In [3]:
def sort_ngrams_by_frequency(ngram_frequencies):
    """Convert n-gram frequencies to a ranked list, most common first."""
    return [ngram for ngram, count in ngram_frequencies.most_common()]

def calculate_out_of_place_distance(document_profile, reference_profile):
    """Calculate the 'out-of-place' distance measure between two n-gram profiles."""
    max_distance = len(reference_profile)  # Maximum distance for n-grams not found in the reference profile
    total_distance = 0
    for position, ngram in enumerate(document_profile):
        if ngram in reference_profile:
            total_distance += abs(position - reference_profile.index(ngram))
        else:
            total_distance += max_distance  # Add maximum distance if n-gram is not found
    return total_distance

# Generate ranked n-gram profiles for each language
ranked_language_profiles = {language: sort_ngrams_by_frequency(profile) for language, profile in language_profiles.items()}

# Example usage with a new document
test_text = "This is a test text to see which language it fits best. I hope it works. I am not sure."
test_ngram_profile = compute_text_profile(test_text, n)
ranked_test_ngram_profile = sort_ngrams_by_frequency(test_ngram_profile)

# Calculate distances from the new text to each language profile
distance_results = {}
for language, ranked_profile in ranked_language_profiles.items():
    distance_results[language] = calculate_out_of_place_distance(ranked_test_ngram_profile, ranked_profile)
    print(f"Distance from {language} profile: {distance_results[language]}")

# Determine the closest language based on the smallest distance
predicted_language = min(distance_results, key=distance_results.get)
print(f"The text is most likely in: {predicted_language}")


Distance from English profile: 78759
Distance from Slovenian profile: 84439
Distance from German profile: 80177
Distance from Spanish profile: 84621
Distance from Croatian profile: 83438
Distance from Turkey profile: 84973
The text is most likely in: English


In [8]:
def analyze_text_file(filepath, n):
    """Analyze a text file and generate its n-gram profile."""
    try:
        with open(filepath, 'r', encoding='utf-8') as file:
            text = file.read()
        return compute_text_profile(text, n)
    except Exception as e:
        print(f"Error reading {filepath}: {e}")
        return None

def evaluate_language_classification(test_dir, language_files, ranked_language_profiles):
    """Evaluate language classification accuracy with test files."""
    classification_results = {}
    for language, filename in language_files.items():
        test_file_path = os.path.join(test_dir, filename)
        print(f"Testing with {test_file_path}...")
        test_profile = analyze_text_file(test_file_path, n)
        if test_profile:
            ranked_test_profile = sort_ngrams_by_frequency(test_profile)
            distances = {lang: calculate_out_of_place_distance(ranked_test_profile, profile) for lang, profile in ranked_language_profiles.items()}
            predicted_language = min(distances, key=distances.get)
            classification_results[language] = predicted_language
            correctness = "CORRECT" if language == predicted_language else "INCORRECT"
            print(f"{correctness}: Tested text: {language}. Classified language: {predicted_language}. Distance: {distances[predicted_language]}")
        else:
            print(f"Failed to process test file for {language}.")
    return classification_results

# Directory containing test texts
test_files_directory = 'C:/Program Files/Visual Studio Code Projects/Language Technology Lab Works/Lab 3/testtxts'
language_files = {
    'English': 'testenglish.txt',
    'Spanish': 'testspanish.txt',
    'German': 'testgerman.txt',
    'Croatian': 'testcroatian.txt',
    'Slovenian': 'testslovenian.txt',
    'Turkey': 'testturkish.txt'
}

# Call the evaluation function
classification_results = evaluate_language_classification(test_files_directory, language_files, ranked_language_profiles)


Testing with C:/Program Files/Visual Studio Code Projects/Language Technology Lab Works/Lab 3/testtxts\testenglish.txt...
CORRECT: Tested text: English. Classified language: English. Distance: 1424097
Testing with C:/Program Files/Visual Studio Code Projects/Language Technology Lab Works/Lab 3/testtxts\testspanish.txt...
CORRECT: Tested text: Spanish. Classified language: Spanish. Distance: 1311408
Testing with C:/Program Files/Visual Studio Code Projects/Language Technology Lab Works/Lab 3/testtxts\testgerman.txt...
CORRECT: Tested text: German. Classified language: German. Distance: 2506547
Testing with C:/Program Files/Visual Studio Code Projects/Language Technology Lab Works/Lab 3/testtxts\testcroatian.txt...
CORRECT: Tested text: Croatian. Classified language: Croatian. Distance: 1628843
Testing with C:/Program Files/Visual Studio Code Projects/Language Technology Lab Works/Lab 3/testtxts\testslovenian.txt...
CORRECT: Tested text: Slovenian. Classified language: Slovenian. Distanc